In [52]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [53]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [54]:
df = pd.read_csv('RA.csv')
df.head()

,Região administrativa,Num de veiculos,% de veiculos,Aluguel medio,Dist centro
0,Plano Piloto,74038,86.9,2268.0,0
1,Gama,25485,65.0,965.0,30
2,Taguatinga,45951,72.0,1037.0,19
3,Brazlândia,8798,56.1,850.0,59
4,Sobradinho,13364,72.9,1003.0,22


## Variáveis do problema

In [55]:
N = len(df)                             # Número de RAs
V = df['Num de veiculos'].to_numpy()    # Número de veículos
D = df['Dist centro'].to_numpy()        # Distância entre a RA e o centro de distribuição das vacinas (plano piloto)

cities = df['Região administrativa'].to_numpy()

In [56]:
cap_min = 10000         # Capacidade min de atendimento de cada drive thru
cap_max = 20000         # Capacidade max de atendimento de cada drive thru
doses = 200000          # Número de doses de vacina disponíveis
qtde_pessoas = 2        # Quantidade média de pessoas vacinadas por carro
dist_max = 15           # Distância máxima entre dois postos drive thru

## Variável alvo

Quantidade de drive thrus por Região Administrativa do DF

- **Limite mínimo**: 0
- **Limite máximo**: 2

In [57]:
X = []

for i in range(N):
    qt_vacinantes = V[i] * qtde_pessoas
    qt_max = int(min(cap_max, qt_vacinantes))

    if qt_vacinantes < cap_min:
        X.append(solver.IntVar(0, 0, f'X{i}'))
    else:
        X.append(solver.IntVar(0, qt_max, f'X{i}'))

## Restrições

1. O número máximo de doses distribuídas deve ser menor do que a qtde disponível


$$ \sum_{i=0}^{N} X_{i}  = doses$$

In [58]:
ct = solver.Constraint(doses, doses, 'ct_num_doses')
for i in range(N):
    ct.SetCoefficient(X[i], 1)

2. Uma região só receberá um drive thru se a quantidade de vacinantes (Qtde de carros [V] * média de vacinantes por carro [q]) for maior do que a capacidade de vacinação do posto

$$\sum X_{i} \frac{c}{qV_{i}} \leq 1$$

In [59]:
# ct = solver.Constraint(0, qtde_pessoas/cap_max, 'ct_num_vacinantes')
# for i in range(N):
#     ct.SetCoefficient(X[i], 1/V[i])

## Função objetivo

Minimizar o custo de distribuição das vacinas

Aqui, é considerado o custo de transporte das vacinas até o posto (considerando que o centro de distribuição está localizado no Plano Piloto) e o custo de armazenamento das vacinas (considerando o preço do aluguel médio de cada região administrativa)

$$min \left ( \sum_{i=0}^{N} X_{i} (A_{i} + gD_{i}) \right )$$

In [60]:
objective = solver.Objective()

for i in range(N):
    objective.SetCoefficient(X[i], 1)
objective.SetMaximization()

## Resultado

In [61]:
status = solver.Solve()

In [62]:
if status == pywraplp.Solver.OPTIMAL:
    print('Valor objetivo =', solver.Objective().Value())
    for i in range(N):
        n = X[i].solution_value()
        if n:
            print(f"{cities[i]}: {n} vacinas")
else:
    print('O problema não possui solução ótima')

Valor objetivo = 200000.0
Recanto das Emas: 20000.0 vacinas
Riacho Fundo: 17936.0 vacinas
Lago Norte: 19858.0 vacinas
Águas Claras: 20000.0 vacinas
Riacho Fundo II: 20000.0 vacinas
Sudoeste/Octogonal: 20000.0 vacinas
Park Way: 10194.0 vacinas
Sobradinho II: 20000.0 vacinas
Jardim Botânico: 13922.0 vacinas
Itapoã: 19788.0 vacinas
Vicente Pires: 18302.0 vacinas
